# Evaluator with more manual steps

In [1]:
%%capture
#########################
# Requirements and Imports
#########################
from os import path
import ipywidgets as widgets
from pathlib import Path

import torch
import numpy as np
import random

## Reproducibility

In [2]:
reproducibility_field = widgets.Checkbox(
    value = True,
    description = "Reproducibility",
    disabled = False
)
display(reproducibility_field)

Checkbox(value=True, description='Reproducibility')

In [3]:
reproducibility = reproducibility_field.value
if reproducibility:
    seed_field = widgets.IntText(
        value=0,
        placeholder=0,
        description="Seed:",
        disabled=False
    )
    display(seed_field)

IntText(value=0, description='Seed:')

In [23]:
# Set seeds for PRGs

reproducibility = reproducibility_field.value
if reproducibility:
    seed = seed_field.value
    print(seed)
    
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    # Use deterministic Algorithms
    torch.use_deterministic_algorithms(True)
    torch.utils.deterministic.fill_uninitialized_memory = True

0


## Game Selection

In [5]:
import ocatari
all_games = ocatari.core.AVAILABLE_GAMES
print(all_games)

['Adventure', 'AirRaid', 'Alien', 'Amidar', 'Assault', 'Asterix', 'Asteroids', 'Atlantis', 'BankHeist', 'BattleZone', 'BeamRider', 'Berzerk', 'Bowling', 'Boxing', 'Breakout', 'Carnival', 'Centipede', 'ChopperCommand', 'CrazyClimber', 'DemonAttack', 'DonkeyKong', 'DoubleDunk', 'Enduro', 'FishingDerby', 'Freeway', 'Frogger', 'Frostbite', 'Galaxian', 'Gopher', 'Hero', 'IceHockey', 'Jamesbond', 'Kangaroo', 'KeystoneKapers', 'KingKong', 'Krull', 'KungFuMaster', 'MarioBros', 'MontezumaRevenge', 'MsPacman', 'NameThisGame', 'Pacman', 'Phoenix', 'Pitfall', 'Pong', 'Pooyan', 'PrivateEye', 'Qbert', 'Riverraid', 'RoadRunner', 'Seaquest', 'Skiing', 'SpaceInvaders', 'StarGunner', 'Tennis', 'TimePilot', 'UpNDown', 'Venture', 'VideoPinball', 'YarsRevenge', 'Zaxxon']


In [6]:
hack_games = [x.name.replace('.py', '') for x in Path('../hackatari/games').glob('*.py') if x.is_file() and x.name != '__init__.py']

games_option = []
for g in all_games:
    if g.lower() in hack_games:
        games_option.append(g)


game_selector = widgets.Select(
    options = games_option,
    value = "Pong",
    description = 'Game:',
    disabled = False,
)
game_selector.layout.width = '30%'
display(game_selector)

Select(description='Game:', index=21, layout=Layout(width='30%'), options=('Amidar', 'Assault', 'Asterix', 'At…

In [17]:
selected_game = game_selector.value
selected_game

'Seaquest'

# Model Selection

In [ ]:
from IPython.display import display, clear_output
import os

model_dir_input = widgets.Text(
    value='../models',
    placeholder='Enter relative path...',
    description='Relative Path:',
    disabled=False,
    layout={'width': '30%'}
)

output_area = widgets.Output() #create an output area for feedback

def on_button_click(b):
    global stored_path_object
    with output_area: #capture output into the defined output area
        clear_output() #clear previous output
        relative_path = model_dir_input.value
        
        if os.path.exists(relative_path):
            print(f"Path '{relative_path}' exists. Path is stored.")
            stored_path_object = Path(relative_path) #create path object
        else:
            print(f"Path '{relative_path}' does not exist. Path is set to None.")
            stored_path_object = None
              

button = widgets.Button(description="Verify Model Path")
button.on_click(on_button_click)

input_and_button = widgets.HBox([model_dir_input, button]) # create hbox

# Display the widget
display(input_and_button, output_area)


Output()

In [43]:
models = []
for model_pattern in [f'{selected_game}/**/*.gz']: # seems not support f'{selected_game}/**/*.*model' yet
    models.extend(stored_path_object.glob(model_pattern))

if len(models) == 0:
    print(f"No models found for game '{selected_game}'.")
else:
    models.sort()
    model_selector = widgets.SelectMultiple(
        options=models,
        value=[],
        description="Select Models",
        display="flex",
        flew_flow="column",
        align_items="stretch",
        style={"description_width": "initial"},
        disabled=False
    )
    model_selector.layout.width = '30%'
    display(model_selector)


SelectMultiple(description='Select Models', layout=Layout(width='30%'), options=(PosixPath('../models/Seaquest…

In [45]:
models_list = list(model_selector.value)
assert models_list, "Please select at least one model"
models_list

[PosixPath('../models/Seaquest/1/mdqn_modern_50M.gz')]

# Setup commands arguments

### Load and select available modfications

In [46]:
import hackatari.core as core

avail_mod_for_game = core._available_modifications(selected_game).split('*')[1:]
modifies_list = [mod.split(':') for mod in avail_mod_for_game]

modifies_selector = widgets.VBox([
        widgets.HBox([
            widgets.Label(value=mod_name, layout={'width': '10%'}),
            widgets.Label(value=mod_info, layout={'width': '50%'}),
            widgets.Dropdown(options=[True, False], value=False, description='Value:',layout={'width': '15%'}, justify_content = 'flex-end')
        ], justify_content = 'space-between') for mod_name, mod_info in modifies_list
    ], layout={'width': '75%'})

display(modifies_selector)

In [47]:
mod_args = ' '.join([f'{mod.children[0].value.strip()}' for mod in modifies_selector.children if mod.children[2].value])
if mod_args:
    mod_args = f'-m {mod_args}'

### Setup other arguments for executing commands

In [48]:
from ocatari.core import OCAtari
# parameters: difficulty and game_mode
g = OCAtari(selected_game)

game_text = widgets.Label  (
    value=f"For the selected game '{selected_game}':",
    disabled=False
)

mode_dropdown = widgets.Dropdown(
    options=g.env.env.ale.getAvailableModes(),
    value=0,
    description='game mode:',
    disabled=False,
    layout = {'width':'10%'}
)

difficulty_dropdown = widgets.Dropdown(
    options=g.env.env.ale.getAvailableDifficulties(),
    value=0,
    description='difficulty:',
    disabled=False,
    layout = {'width':'10%'}
)

g.close()


dp_dropdown = widgets.Dropdown(
    options=[True, False],
    value=False,
    description='dopamine_pooling:',
    style={'description_width': 'initial'},
    #layout=widgets.Layout(width='300px'),
    disabled=False,
    layout = {'width':'15%'}
)

window_inttext = widgets.IntText(
    value=4,
    description='window_size:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='120px'),
    disabled=False,
    #layout = {'width':'10%'}
)

frameskip_inttext = widgets.IntText(
    value=4,
    description='frameskip:',
    disabled=False,
    layout=widgets.Layout(width='120px'),
    #layout = {'width':'10%'}
)

has_dqn_model = any([ ("dqn" in model.as_posix() or "c51" in model.as_posix()) for model in models_list])

obs_dropdown = widgets.Dropdown(
    options=['dqn'] if has_dqn_model else ['ori', 'obj'],
    value='dqn',
    description='obs_mode:',
    disabled=False,
    layout = {'width':'10%'}
)

episodes_inttext = widgets.IntText(
    value=10,
    description='num_episodes:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='150px'),
    disabled=False,
    #layout = {'width':'10%'}
)

parameters_selector =  widgets.VBox( [game_text, 
                                     widgets.HBox([mode_dropdown, difficulty_dropdown, dp_dropdown, window_inttext, frameskip_inttext, obs_dropdown, episodes_inttext],
                                                  justify_content = 'space-between', layout = {'width':'100%'})
])

# Display the widgets
display(parameters_selector)



In [49]:
stored_parameter = ""
stored_parameter += f" -mo {mode_dropdown.value}"
stored_parameter += f" -d {difficulty_dropdown.value}"
stored_parameter += " -dp" if dp_dropdown.value else ""
stored_parameter += f" -w {window_inttext.value}"
stored_parameter += f" -f {frameskip_inttext.value}"
stored_parameter += f" -obs {obs_dropdown.value}" if obs_dropdown.value else ""

print(stored_parameter)

eval_parameter = f" -e {episodes_inttext.value}"

 -mo 0 -d 0 -w 4 -f 4 -obs dqn


# Run Game

In [52]:
## prepare for the command
eval_script = Path(r"scripts/eval.py")
run_script = Path(r"scripts/run.py")

notebook_dir = path.abspath("")
project_root = path.dirname(notebook_dir)
print(f"Project Root: {project_root}")

# use posix_paths for compatibile with different OS
eval_path = (Path(project_root) / eval_script).as_posix()
run_path = (Path(project_root) / run_script).as_posix()

Project Root: /mnt/c/Users/peng_/workspace/HackAtari


### Execuate the command with eval.py

In [ ]:
# "eval.py" can accept a list of models
models_path = ' '.join( f'{model.as_posix()}' for model in models_list )


eval_command = f"python {eval_path} -g {selected_game} -a {models_path} {mod_args} {stored_parameter} {eval_parameter}"

print("eval command: " + eval_command)

!{eval_command}

Project Root: /mnt/c/Users/peng_/workspace/HackAtari
eval command: python /mnt/c/Users/peng_/workspace/HackAtari/scripts/eval.py -g Seaquest -a ../models/Seaquest/1/mdqn_modern_50M.gz   -mo 0 -d 0 -w 4 -f 4 -obs dqn  -e 10
A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
Runing for episodes: 10
Loaded agent from ../models/Seaquest/1/mdqn_modern_50M.gz
Episode 1: Reward = 13650.0, Time = 3.08 seconds with 2407 steps and actions: {0: 47, 1: 39, 2: 110, 3: 68, 4: 7, 5: 8, 6: 188, 7: 142, 8: 185, 9: 118, 10: 22, 11: 148, 12: 39, 13: 32, 14: 202, 15: 417, 16: 324, 17: 311}
Episode 2: Reward = 6530.0, Time = 2.82 seconds with 2554 steps and actions: {0: 54, 1: 68, 2: 104, 3: 10, 4: 8, 5: 3, 6: 144, 7: 168, 8: 158, 9: 100, 10: 9, 11: 124, 12: 5, 13: 21, 14: 231, 15: 497, 16: 495, 17: 355}
Episode 3: Reward = 16930.0, Time = 3.44 seconds with 3158 steps and actions: {0: 53, 1: 42, 2: 142, 3: 78, 4: 12, 5: 25, 6: 229, 7: 202, 8: 197, 9: 171, 10: 25, 11: 161, 12: 3

### Execute the command with run.py (accept only one model and visualize each espiode)

In [ ]:
# "run.py" can only accept one model
assert len(models_list) > 0, "Please select at least one model"
models_path = f'{models_list[0].as_posix()}'

run_command = f"python {run_path} -g {selected_game} -a {models_path} {mod_args} {stored_parameter}"
print("run command: " + run_command)

!{run_command}

run command: python /mnt/c/Users/peng_/workspace/HackAtari/scripts/run.py -g Seaquest -a ../models/Seaquest/1/mdqn_modern_50M.gz   -mo 0 -d 0 -w 4 -f 4 -obs dqn


1706.33s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


A.L.E: Arcade Learning Environment (version 0.10.2+c9d4b19)
[Powered by Stella]
Loaded agent from ../models/Seaquest/1/mdqn_modern_50M.gz


# Evaluation (with AIM-UI trace)

In [58]:
# add current module to the path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

### Experiments with all subsets of selected modifs

In [59]:
from itertools import combinations
mod_list = mod_args.split(" ")

# return all non-emtpy subsets
def subsets(fullset):    
    result = []
    for i in range(1, len(fullset) + 1):
        tmp = [ list(c) for c in combinations(fullset, i) ]
        result += tmp
        
    return result

mod_sublists = subsets(mod_list)

print(mod_sublists)

[['']]


In [60]:
%reload_ext aim

!aim init -y # it will (re-)initialize an empty aim repo.

%aim up

2502.87s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Re-initialized empty Aim repository at /mnt/c/Users/peng_/workspace/HackAtari/automatic_evaluator


In [61]:
%reload_ext autoreload

%autoreload 1

%aimport utils

# agent_paths = [model.as_posix() for model in models_list]
# for agent_path in agent_paths:
#     agent_name = "/".join(agent_path.split("/")[-2:])
#     print(agent_name)
        
for modifs in mod_sublists:
    utils.eval_withAimRun(selected_game, [ model.as_posix() for model in models_list], modifs)


Loaded agent from ../models/Seaquest/1/mdqn_modern_50M.gz
--------------------------------------
Done.
